In [1]:
import requests
import pandas as pd
import os

In [2]:
# use this to offset response
# url = "https://api.reliefweb.int/v1/sources?appname=rwint-user-0&profile=list&preset=latest&slim=1&query%5Bvalue%5D=status%3Aactive&query%5Boperator%5D=AND&offset=1000&limit=1000"

In [3]:
# create a list for sahel countries
sahel_countries = ["Djibouti", "Eritrea", "Ethiopia", "Sudan", "Chad", "Niger", "Nigeria", "Mali", "Burkina%20Faso", "Mauitania", "Senegal"]

In [4]:
# get countries from hotspots data
hotspots = pd.read_csv("infant_hotspots.csv")

# remove space so we can use for URL generator
hotspots["ADM0_NAME"]=hotspots["ADM0_NAME"].str.replace(" ", "%20")

In [5]:
# create a list of countries
country_list = list(hotspots["ADM0_NAME"].unique())

# check if sahel countries are included in country_list. If not, append to country_list
for country in sahel_countries:
    if country not in country_list:
        country_list.append(country)

In [29]:
country_list.sort()

In [30]:
country_list

['Angola',
 'Burkina%20Faso',
 'Central%20African%20Republic',
 'Chad',
 "Côte%20d'Ivoire",
 'Democratic%20Republic%20of%20the%20Congo',
 'Djibouti',
 'Eritrea',
 'Ethiopia',
 'Guinea',
 'Liberia',
 'Mali',
 'Mauitania',
 'Niger',
 'Nigeria',
 'Senegal',
 'Sierra%20Leone',
 'Somalia',
 'South%20Sudan',
 'Sudan',
 'Zambia']

In [6]:
country_list

['Sierra%20Leone',
 'South%20Sudan',
 'Burkina%20Faso',
 'Central%20African%20Republic',
 'Mali',
 'Somalia',
 "Côte%20d'Ivoire",
 'Guinea',
 'Chad',
 'Zambia',
 'Nigeria',
 'Democratic%20Republic%20of%20the%20Congo',
 'Angola',
 'Liberia',
 'Djibouti',
 'Eritrea',
 'Ethiopia',
 'Sudan',
 'Niger',
 'Mauitania',
 'Senegal']

### custom function for building API query

In [7]:
def url_builder(country):
    url = "https://api.reliefweb.int/v1/sources?appname=rwint-user-0&profile=list&preset=latest&slim=1&query%5Bvalue%5D=%28country.exact%3A%22"+ country +"%22%29+AND+status%3Aactive&query%5Boperator%5D=AND"
    return url

### get request for all countries in sahel region

In [12]:
orgs_list = []
for country in country_list:
    url = url_builder(country)
    response = requests.get(url)
    response_json = response.json()
    for item in response_json["data"]:
        orgs_list.append(item["fields"])

In [13]:
# make into dataframe
df = pd.DataFrame(orgs_list)

In [14]:
df.shape

(126, 5)

In [15]:
# filter out government agencies
orgs_df = df[~df["name"].str.contains("Government")]

In [16]:
orgs_df

,name,status,shortname,homepage,url
0,UN Country Team in Sierra Leone,active,UNCT Sierra Leone,https://sierraleone.un.org/,https://reliefweb.int/taxonomy/term/15192
1,Sierra Leone Red Cross Society,active,Sierra Leone Red Cross,http://sierraleoneredcross.org,https://reliefweb.int/taxonomy/term/12115
3,HPF South Sudan,active,HPF South Sudan,https://hpfsouthsudan.org,https://reliefweb.int/taxonomy/term/51020
4,ChildBride Solidarity,active,CBS,http://childbridesolidarity.org/,https://reliefweb.int/taxonomy/term/50693
5,Conflict Sensitivity Resource Facility,active,CSRF,https://www.csrf-southsudan.org/,https://reliefweb.int/taxonomy/term/50492
...,...,...,...,...,...
120,UN Integrated Strategy for the Sahel,active,UNISS,https://unis-sahel.org,https://reliefweb.int/taxonomy/term/51412
121,Mo Ibrahim Foundation,active,MIF,http://mo.ibrahim.foundation/,https://reliefweb.int/taxonomy/term/49058
122,UN Office for West Africa and the Sahel,active,UNOWAS,http://unowas.unmissions.org/,https://reliefweb.int/taxonomy/term/37996
123,UN Resident Coordinator in Senegal,active,UN RC Senegal,https://senegal.un.org/fr/about/about-the-resi...,https://reliefweb.int/taxonomy/term/22451


In [17]:
orgs_df.to_csv("org_names.csv", index = False)

### Get coordinates from google maps

In [21]:
def search_google_maps(query):
    base_url = "https://www.google.com/maps/search/"
    search_url = f"{base_url}?q={query}"
    
    response = requests.get(search_url)
    return response.text

In [22]:
def get_coordinates(query):
    location = search_google_maps(query)
    index_key = "APP_INITIALIZATION_STATE=[[["
    index = location.find(index_key)
    if index != -1:
        # Extract the 55 characters after the found index
        extracted_text = location[index + len(index_key): index + len(index_key) + 55]
        split_text = extracted_text.split(",")
    else:
        split_text = "Location not found"
    return split_text

In [23]:
org_locations = []
for org in orgs_df["name"].unique():
    location = get_coordinates(org)
    org_loc = {
        "name": org,
        "lat": location[1:][1].replace("]", ""),
        "long": location[1:][0]
    }
    org_locations.append(org_loc)

In [26]:
orgs_df = orgs_df.merge(pd.DataFrame(org_locations), how = "left", on = ["name"])

In [27]:
orgs_df.tail()

,name,status,shortname,homepage,url,lat,long
104,UN Integrated Strategy for the Sahel,active,UNISS,https://unis-sahel.org,https://reliefweb.int/taxonomy/term/51412,12.06342830137239,-26.354907731243127
105,Mo Ibrahim Foundation,active,MIF,http://mo.ibrahim.foundation/,https://reliefweb.int/taxonomy/term/49058,37.599999999999994,-95.665
106,UN Office for West Africa and the Sahel,active,UNOWAS,http://unowas.unmissions.org/,https://reliefweb.int/taxonomy/term/37996,14.75003498970745,-17.511238685157934
107,UN Resident Coordinator in Senegal,active,UN RC Senegal,https://senegal.un.org/fr/about/about-the-resi...,https://reliefweb.int/taxonomy/term/22451,14.73964978971402,-17.517445985158115
108,Croix-Rouge Sénégalaise,active,Croix-Rouge Sénégalaise,https://www.ifrc.org/fr/national-societies-dir...,https://reliefweb.int/taxonomy/term/11226,14.67969579412538,-17.47053482721495


In [28]:
orgs_df.to_csv("org_locations.csv", index = False)

In [1]:
orgs_df.shape

NameError: name 'orgs_df' is not defined